In [20]:
import pandas as pd
import numpy as np


In [21]:
def process_base_hand_data(path_to_data: str) -> pd.DataFrame:
    raw_hand_data = pd.read_csv(path_to_data)
    raw_hand_data = raw_hand_data.rename(columns={
        'equity.flop': 'flop_equity',
        'equity.turn': 'turn_equity',
        'equity.river': 'river_equity'})
    raw_hand_data = raw_hand_data.assign(
        equity_delta=raw_hand_data['river_equity']-raw_hand_data['flop_equity']
        )
    return raw_hand_data[
        ['hand','connectedness','suited','pair','flop_equity','turn_equity','river_equity','equity_delta']
        ]

    

In [22]:
ev_data = pd.read_csv("data/raw/ev_pca_data.csv").set_index('hand')
dom_block_data = pd.read_csv("data/raw/dom_block_data.csv").set_index('hand')
hand_data = process_base_hand_data("data/raw/hand_data.csv")
hand_data['high_card'] = hand_data['hand'].apply(lambda x: '23456789TJQKA'.index(x[0]))
hand_data['low_card'] = hand_data['hand'].apply(lambda x: '23456789TJQKA'.index(x[1]))
hand_data['hand_type'] = np.select(
    condlist=[
        (hand_data['pair'] == True),
        (hand_data['suited'] == False),
        (hand_data['suited'] == True),
    ],
    choicelist=['pair', 'offsuit', 'suited'],
    default='other'
)
hand_data.set_index("hand", inplace=True)

In [23]:
full_data = pd.concat([hand_data, dom_block_data, ev_data], axis=1)

In [25]:
full_data.reset_index().to_csv('data/processed/full_data.csv', index=False)